In [1]:
import logging
import sys
import json
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score,roc_curve, auc, confusion_matrix,classification_report

def read_answers(filename):
    answers={}
    with open(filename) as f:
        for line in f:
            line=line.strip()
            js=json.loads(line)
            answers[js['idx']]=js['target']
    return answers

def read_predictions(filename):
    predictions={}
    with open(filename) as f:
        for line in f:
            line=line.strip()
            idx,label=line.split()
            predictions[int(idx)]=int(label)
    return predictions

def read_predictions_prob(filename):
    predictions_prob={}
    with open(filename) as f:
        for line in f:
            line=line.strip()
            idx,label=line.split()
            predictions_prob[int(idx)]= float(label)
    return predictions_prob

def calculate_scores(answers,predictions,predictions_prob):
    Acc=[]
    Ans=[]
    Pred=[]
    Pred_prob=[]
    for key in answers:
        Ans.append(answers[key])
        if key not in predictions:
            logging.error("Missing prediction for index {}.".format(key))
            sys.exit()
        Acc.append(answers[key]==predictions[key])
    for key in predictions:
        Pred.append(predictions[key])
    for key in predictions_prob:
        Pred_prob.append(predictions_prob[key])
    scores={}
    results = []
#     scores['acc']=np.mean(Acc)
    fpr, tpr, _ = roc_curve(Ans, Pred_prob)
    results.append(auc(fpr, tpr)*100)
    results.append(accuracy_score(Ans,Pred)*100)
    results.append(recall_score(Ans,Pred)*100)
    results.append(precision_score(Ans,Pred,zero_division=0)*100)
    results.append(f1_score(Ans,Pred,zero_division=0)*100)
    zipped_result = zip(Ans,Pred,Pred_prob)
    sorted_zip = sorted(zipped_result, key=lambda x: x[2],reverse=True)
    print(confusion_matrix(Ans,Pred))
#     print('auc\t',auc(fpr, tpr))
#     print('acc\t',accuracy_score(Ans,Pred))
#     print('f1\t',f1_score(Ans,Pred))
#     print('recall\t',recall_score(Ans,Pred))
#     print('precision\t',precision_score(Ans,Pred))
    print(results)
    return results,sorted_zip,Pred_prob


In [2]:
def read_output(test_dir,result_dir):
    answers=read_answers(test_dir+'test.jsonl')
    predictions=read_predictions(result_dir+'predictions.txt')
    predictions_prob = read_predictions_prob(result_dir+'predictions_prob.txt')
    scores,sorted_zip, Pred_prob=calculate_scores(answers,predictions,predictions_prob)
    return scores,sorted_zip,Pred_prob

In [3]:
result_list = [] 
for i in range(0,19):
    try:
        test_dir = f'../msr_dataset/ros_msr/data_split_{i}/'
        result_dir = f'msr_output/ros_msr/saved_models_{i}/'
        print(result_dir)
        result,sorted_zip,pred_prob = read_output(test_dir,result_dir)
        # import pickle
        # pickle.dump(sorted_zip, open( f'../../metrics_exp/data/Codebert_model/reveal/oss/{i}_zip_ans_pred_prob.pkl', "wb" ))
        # result_list.append(result)
    except:
        print('error',i)

msr_output/ros_msr/saved_models_0/
[[34518  1005]
 [ 1384   821]]
[76.60132239704429, 93.66783290924512, 37.23356009070295, 44.961664841182916, 40.734309104440584]
msr_output/ros_msr/saved_models_1/
[[34550   952]
 [ 1402   824]]
[80.29656200986969, 93.76060220525869, 37.01707097933513, 46.3963963963964, 41.17941029485258]
msr_output/ros_msr/saved_models_2/
[[34610   964]
 [ 1389   765]]
[74.40449763551453, 93.76325275657337, 35.515320334261844, 44.24522845575477, 39.40252382178728]
msr_output/ros_msr/saved_models_3/
[[34615   941]
 [ 1361   811]]
[80.82965498670217, 93.89843087362172, 37.338858195211785, 46.289954337899545, 41.335372069317025]
msr_output/ros_msr/saved_models_4/
[[34665   864]
 [ 1394   805]]
[70.99250026920473, 94.01505513146734, 36.607548885857206, 48.23247453565009, 41.623578076525334]
msr_output/ros_msr/saved_models_5/
[[34676   888]
 [ 1400   764]]
[74.2297515858006, 93.93553859202714, 35.304990757855826, 46.246973365617436, 40.04192872117401]
msr_output/ros_msr/s

In [3]:
i=0
test_dir = f'../msr_dataset/origin/data_split_{i}/'
result_dir = f'msr_outout/ros_4x/'
result,sorted_zip,pred_prob_2 = read_output(test_dir,result_dir)

[[34612   911]
 [ 1380   825]]
[70.96231479295169, 93.92758693808312, 37.41496598639456, 47.52304147465438, 41.86754630804364]


In [ ]:
import seaborn as sns
sns.displot(pred_prob, kde=False, 
             bins=100, color = 'blue')

In [ ]:
import seaborn as sns
sns.displot(pred_prob_2, kde=False, 
             bins=100, color = 'blue')

In [ ]:
import pandas as pd
df = pd.DataFrame(result_list, columns = ['auc', 'acc','recall','precision','f1'])